<a href="https://colab.research.google.com/github/zahaanghadiali/ML-projects/blob/main/Movie_Lens_Collaborative_Filtering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from scipy import sparse

In [ ]:
ratings = pd.read_csv('dataset/ratings.csv')
movies = pd.read_csv('dataset/movies.csv')
ratings = pd.merge(movies,ratings).drop(['genres','timestamp'],axis=1)
print(ratings.shape)
ratings.head()

(69503, 4)


,movieId,title,userId,rating
0,1,Toy Story (1995),1,4.0
1,1,Toy Story (1995),5,4.0
2,1,Toy Story (1995),7,4.5
3,1,Toy Story (1995),15,2.5
4,1,Toy Story (1995),17,4.5


In [ ]:
userRatings = ratings.pivot_table(index=['userId'],columns=['title'],values='rating')
userRatings.head()
print("Before: ",userRatings.shape)
userRatings = userRatings.dropna(thresh=10, axis=1).fillna(0,axis=1)
#userRatings.fillna(0, inplace=True)
print("After: ",userRatings.shape)

Before:  (448, 7761)
After:  (448, 1716)


In [ ]:
corrMatrix = userRatings.corr(method='pearson')
corrMatrix.head(100)

title,"'burbs, The (1989)",(500) Days of Summer (2009),10 Cloverfield Lane (2016),10 Things I Hate About You (1999),"10,000 BC (2008)",101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),12 Angry Men (1957),12 Years a Slave (2013),127 Hours (2010),13 Going on 30 (2004),"13th Warrior, The (1999)",1408 (2007),2 Days in the Valley (1996),"2 Fast 2 Furious (Fast and the Furious 2, The) (2003)","20,000 Leagues Under the Sea (1954)",2001: A Space Odyssey (1968),2010: The Year We Make Contact (1984),2012 (2009),21 (2008),21 Grams (2003),21 Jump Street (2012),22 Jump Street (2014),25th Hour (2002),27 Dresses (2008),28 Days (2000),28 Days Later (2002),28 Weeks Later (2007),300 (2007),"39 Steps, The (1935)",3:10 to Yuma (2007),40 Days and 40 Nights (2002),"40-Year-Old Virgin, The (2005)",50 First Dates (2004),50/50 (2011),"6th Day, The (2000)",8 1/2 (8½) (1963),8 Mile (2002),8MM (1999),"A-Team, The (2010)",...,Willy Wonka & the Chocolate Factory (1971),Winnie the Pooh and the Blustery Day (1968),Witness (1985),"Wizard of Oz, The (1939)",Wolf (1994),"Wolf of Wall Street, The (2013)",Wonder Boys (2000),Working Girl (1988),"World Is Not Enough, The (1999)",World War Z (2013),"World's End, The (2013)",Wreck-It Ralph (2012),"Wrestler, The (2008)",Wyatt Earp (1994),"X-Files: Fight the Future, The (1998)",X-Men (2000),X-Men Origins: Wolverine (2009),X-Men: Days of Future Past (2014),X-Men: First Class (2011),X-Men: The Last Stand (2006),X2: X-Men United (2003),Yellow Submarine (1968),Yes Man (2008),Yojimbo (1961),You Don't Mess with the Zohan (2008),You've Got Mail (1998),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),Young Sherlock Holmes (1985),Zack and Miri Make a Porno (2008),Zero Dark Thirty (2012),Zero Effect (1998),Zodiac (2007),Zombieland (2009),Zoolander (2001),Zootopia (2016),eXistenZ (1999),xXx (2002),¡Three Amigos! (1986)
title,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"'burbs, The (1989)",1.000000,0.018051,-0.022243,0.163557,0.027912,0.100255,0.191274,0.009911,-0.025404,-0.025157,0.099924,0.139565,0.100028,0.076722,0.132704,0.231121,0.172870,0.158910,0.023411,-0.027390,0.047854,-0.028138,-0.026180,0.123650,-0.026713,0.012829,0.047438,-0.026820,0.065277,0.006685,0.151307,0.166527,0.142120,0.049379,0.056937,0.017372,0.084486,0.102735,-0.024084,-0.025437,...,0.110669,0.190498,0.090155,0.135271,0.049022,0.004418,0.157925,0.240763,0.111194,-0.026333,0.063735,-0.025190,0.030288,0.071631,0.135476,0.137121,0.024127,-0.027412,0.012447,0.067581,0.094388,0.172997,0.044842,-0.022869,-0.023660,0.108656,0.137945,0.085761,0.103193,0.362541,0.040732,0.063733,0.170994,0.058078,0.015013,0.047768,0.024737,0.108378,0.097597,0.187828
(500) Days of Summer (2009),0.018051,1.000000,0.127534,0.239863,0.232071,0.141723,0.134085,0.160864,0.161255,0.218109,0.321425,0.053551,0.216373,0.019397,0.293916,-0.018358,0.075818,-0.041159,0.332065,0.303686,0.152012,0.272657,0.113274,0.040499,0.265204,0.028250,0.155150,0.196414,0.307295,-0.037325,0.281535,0.199102,0.294015,0.323759,0.418186,-0.011058,-0.000907,0.175500,0.013057,0.190393,...,-0.014272,0.030614,0.024998,0.094916,0.011653,0.254921,0.041870,0.011389,0.043483,0.273744,0.267541,0.144145,0.326903,-0.006695,0.002176,0.154665,0.292442,0.277604,0.328429,0.332424,0.193535,0.013309,0.425170,-0.038483,0.173697,0.093724,-0.005557,0.075019,0.021773,0.038862,0.367998,0.178120,0.081258,0.400012,0.329623,0.244680,0.256068,0.035551,0.286133,0.081617
10 Cloverfield Lane (2016),-0.022243,0.127534,1.000000,0.017238,0.069177,0.027079,-0.004648,0.011142,-0.025166,0.286880,-0.024994,-0.028305,0.158723,-0.022847,0.051732,-0.027640,0.073329,-0.024230,0.163754,0.052840,-0.030175,0.334392,0.234862,-0.025385,-0.026463,-0.031136,0.258681,0.233469,0.129321,-0.021974,-0.031685,0.052357,0.075747,-0.003710,0.060038,0.074250,-0.023173,0.166641,0.064965,0.060704,...,0.016313,-0.021575,-0.034435,-0.056167,-0.023797,0.232339,-0.025129,-0.024653,0.024672,0.3085

In [ ]:
def get_similar(movie_name,rating):
    similar_ratings = corrMatrix[movie_name]*(rating-2.5)
    similar_ratings = similar_ratings.sort_values(ascending=False)
    #print(type(similar_ratings))
    return similar_ratings

In [ ]:
romantic_lover = [("(500) Days of Summer (2009)",5),("Alice in Wonderland (2010)",3),("Aliens (1986)",1),("2001: A Space Odyssey (1968)",2)]
similar_movies = pd.DataFrame()
for movie,rating in romantic_lover:
    similar_movies = similar_movies.append(get_similar(movie,rating),ignore_index = True)

similar_movies.head(10)

,"'burbs, The (1989)",(500) Days of Summer (2009),10 Cloverfield Lane (2016),10 Things I Hate About You (1999),"10,000 BC (2008)",101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),12 Angry Men (1957),12 Years a Slave (2013),127 Hours (2010),13 Going on 30 (2004),"13th Warrior, The (1999)",1408 (2007),2 Days in the Valley (1996),"2 Fast 2 Furious (Fast and the Furious 2, The) (2003)","20,000 Leagues Under the Sea (1954)",2001: A Space Odyssey (1968),2010: The Year We Make Contact (1984),2012 (2009),21 (2008),21 Grams (2003),21 Jump Street (2012),22 Jump Street (2014),25th Hour (2002),27 Dresses (2008),28 Days (2000),28 Days Later (2002),28 Weeks Later (2007),300 (2007),"39 Steps, The (1935)",3:10 to Yuma (2007),40 Days and 40 Nights (2002),"40-Year-Old Virgin, The (2005)",50 First Dates (2004),50/50 (2011),"6th Day, The (2000)",8 1/2 (8½) (1963),8 Mile (2002),8MM (1999),"A-Team, The (2010)",...,Willy Wonka & the Chocolate Factory (1971),Winnie the Pooh and the Blustery Day (1968),Witness (1985),"Wizard of Oz, The (1939)",Wolf (1994),"Wolf of Wall Street, The (2013)",Wonder Boys (2000),Working Girl (1988),"World Is Not Enough, The (1999)",World War Z (2013),"World's End, The (2013)",Wreck-It Ralph (2012),"Wrestler, The (2008)",Wyatt Earp (1994),"X-Files: Fight the Future, The (1998)",X-Men (2000),X-Men Origins: Wolverine (2009),X-Men: Days of Future Past (2014),X-Men: First Class (2011),X-Men: The Last Stand (2006),X2: X-Men United (2003),Yellow Submarine (1968),Yes Man (2008),Yojimbo (1961),You Don't Mess with the Zohan (2008),You've Got Mail (1998),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),Young Sherlock Holmes (1985),Zack and Miri Make a Porno (2008),Zero Dark Thirty (2012),Zero Effect (1998),Zodiac (2007),Zombieland (2009),Zoolander (2001),Zootopia (2016),eXistenZ (1999),xXx (2002),¡Three Amigos! (1986)
0,0.045127,2.500000,0.318835,0.599657,0.580177,0.354307,0.335213,0.402160,0.403137,0.545274,0.803562,0.133876,0.540933,0.048492,0.734791,-0.045894,0.189545,-0.102897,0.830162,0.759214,0.380029,0.681642,0.283184,0.101247,0.663010,0.070624,0.387875,0.491036,0.768238,-0.093313,0.703838,0.497754,0.735038,0.809397,1.045465,-0.027644,-0.002269,0.438749,0.032643,0.475984,...,-0.035681,0.076536,0.062494,0.237290,0.029133,0.637304,0.104675,0.028474,0.108708,0.684359,0.668852,0.360363,0.817258,-0.016739,0.005440,0.386662,0.731106,0.694010,0.821072,0.831060,0.483837,0.033272,1.062924,-0.096208,0.434244,0.234309,-0.013893,0.187548,0.054432,0.097156,0.919995,0.445300,0.203145,1.000031,0.824058,0.611700,0.640169,0.088878,0.715331,0.204043
1,-0.015408,0.206362,0.152484,0.090433,0.096450,0.092619,0.049108,0.112035,0.114222,0.125187,0.010526,0.038209,0.128364,-0.015826,0.108692,0.002000,0.061366,-0.016784,0.175685,0.171707,0.038454,0.134808,0.056259,0.016147,0.086769,0.052189,0.104121,0.174071,0.157300,-0.015221,0.138791,0.106235,0.102261,0.091323,0.171187,0.071031,0.008325,0.107581,0.008241,0.125463,...,0.008205,0.017700,0.004962,0.037638,-0.016484,0.159591,-0.013143,-0.017077,0.062348,0.137233,0.172291,0.119995,0.161276,-0.020415,0.008181,0.087857,0.157282,0.163823,0.158566,0.171120,0.100145,0.027686,0.157974,-0.015693,0.091846,0.093588,0.047795,0.027727,-0.014303,-0.015386,0.161226,0.065500,0.019142,0.203863,0.190817,0.085020,0.092456,0.053286,0.098145,0.019893
2,-0.223249,-0.016665,-0.250756,-0.043140,-0.084107,-0.051385,-0.171935,-0.297568,-0.000930,-0.030356,-0.006932,-0.348059,-0.316070,-0.383130,-0.202839,-0.370006,-0.652776,-0.342278,-0.096421,-0.007926,-0.176078,-0.186776,-0.122855,-0.320515,0.034231,-0.172464,-0.379341,-0.282575,-0.260515,-0.236488,-0.144103,-0.151426,-0.207036,-0.101700,-0.083801,-0.255718,-0.075229,-0.170691,-0.185431,-0.112263,...,-0.321982,-0.283853,-0.369891,-0.362509,-0.055582,-0.019048,-0.182768,-0.124369,-0.368456,-0.277967,-0.279163,-0.158920,-0.240425,-0.083329,-0.492876,-0.401005,-0.300094,-0.116451,-0.167917,-0.174945,-0.384042,-0.240551,-0.002314,-0.163

In [ ]:
similar_movies.sum().sort_values(ascending=False).head(20)

(500) Days of Summer (2009)                    2.651788
Alice in Wonderland (2010)                     1.419542
Silver Linings Playbook (2012)                 1.349752
Up in the Air (2009)                           1.280973
Help, The (2011)                               1.257905
Moneyball (2011)                               1.180398
Yes Man (2008)                                 1.152759
Notebook, The (2004)                           1.120068
Perks of Being a Wallflower, The (2012)        1.107181
Toy Story 3 (2010)                             1.104867
The Hunger Games (2012)                        1.101092
Grand Budapest Hotel, The (2014)               1.091572
Blind Side, The  (2009)                        1.090727
Curious Case of Benjamin Button, The (2008)    1.085469
Marley & Me (2008)                             1.082752
50/50 (2011)                                   1.075647
Definitely, Maybe (2008)                       1.068125
Moonrise Kingdom (2012)                        1

In [ ]:
action_lover = [("Amazing Spider-Man, The (2012)",5),("Mission: Impossible III (2006)",4),("Toy Story 3 (2010)",2),("2 Fast 2 Furious (Fast and the Furious 2, The) (2003)",4)]
similar_movies = pd.DataFrame()
for movie,rating in action_lover:
    similar_movies = similar_movies.append(get_similar(movie,rating),ignore_index = True)

similar_movies.head(10)
similar_movies.sum().sort_values(ascending=False).head(20)

Amazing Spider-Man, The (2012)                           3.172440
Mission: Impossible III (2006)                           2.798394
2 Fast 2 Furious (Fast and the Furious 2, The) (2003)    2.670028
Snakes on a Plane (2006)                                 2.400950
Blade: Trinity (2004)                                    2.379882
Charlie's Angels: Full Throttle (2003)                   2.346399
Fantastic Four (2005)                                    2.312893
Die Another Day (2002)                                   2.309553
S.W.A.T. (2003)                                          2.207226
Van Helsing (2004)                                       2.199664
Transformers (2007)                                      2.171288
Hellboy (2004)                                           2.169606
Mummy Returns, The (2001)                                2.155910
Paycheck (2003)                                          2.137326
Captain America: The First Avenger (2011)                2.090298
Underworld